<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtT5-XL-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Extracting protein sequences' features using ProtT5-XL-BFD pretrained-model</h3>


**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 18.4MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 


In [2]:
from transformers import TFT5Model, T5Tokenizer,T5Config
import numpy as np
import re
import gc

<b>2. Load the vocabulary and ProtT5-XL-BFD Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd", do_lower_case=False )

In [4]:
model = TFT5Model.from_pretrained("Rostlab/prot_t5_xl_bfd", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5Model: ['decoder.embed_tokens.weight', 'lm_head.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFT5Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5Model were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFT5Model for predictions without further training.


In [5]:
gc.collect()

0

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [8]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [9]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [10]:
embedding = model(input_ids, decoder_input_ids=input_ids)

In [11]:
embedding = np.asarray(embedding)

In [12]:
attention_mask = np.asarray(attention_mask)

In [13]:
# For feature extraction we recommend to use the encoder embedding
encoder_embedding = embedding[2]
decoder_embedding = embedding[0]

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-BFD model<b>

In [14]:
features = [] 
for seq_num in range(len(encoder_embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = encoder_embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [15]:
print(features)

[<tf.Tensor: shape=(7, 1024), dtype=float32, numpy=
array([[ 0.29069272, -0.20804921, -0.21934748, ...,  0.33846605,
         0.46031442, -0.16942428],
       [ 0.20362864, -0.12995909, -0.17139861, ...,  0.10642268,
        -0.37738255,  0.05401207],
       [ 0.03736694, -0.085066  , -0.23616509, ...,  0.25152585,
         0.133386  ,  0.03063774],
       ...,
       [ 0.4778799 , -0.23029423, -0.10682346, ...,  0.40571713,
         0.52513707,  0.19175269],
       [ 0.07990415,  0.07094706, -0.08554695, ...,  0.24947524,
         0.3653833 , -0.4550627 ],
       [ 0.09110177,  0.17046821,  0.4207921 , ...,  0.2562663 ,
         0.02010929, -0.11016787]], dtype=float32)>, <tf.Tensor: shape=(5, 1024), dtype=float32, numpy=
array([[ 0.32152942, -0.09309329, -0.15645394, ..., -0.02557713,
         0.06829392, -0.1552771 ],
       [ 0.10843599, -0.06435566,  0.15844908, ...,  0.05684958,
        -0.16503634,  0.15184812],
       [ 0.21142846,  0.11896314,  0.08190019, ...,  0.2332429 ,
  